**Deskripsi Dataset**

Dataset "Car Features and MSRP" merupakan kumpulan data yang berisi berbagai informasi teknis dan spesifikasi mobil yang digunakan untuk memprediksi harga (MSRP - Manufacturer’s Suggested Retail Price) sebuah mobil.

Kolom yang ada pada dataset:


| Kolom                 | Fungsi                                                                                                 |
| --------------------- | ------------------------------------------------------------------------------------------------------ |
| **Make**              | Memberi informasi merek mobil. Bisa memengaruhi persepsi kualitas & harga (fitur kategorikal).         |
| **Model**             | Identifikasi unik tiap mobil. Bisa terlalu spesifik, sering di-*drop* atau di-*encode* jika relevan.   |
| **Year**              | Menggambarkan usia mobil; tahun lebih baru biasanya harga lebih mahal. Penting untuk regresi.          |
| **Engine Fuel Type**  | Tipe bahan bakar bisa pengaruhi efisiensi dan harga (contoh: electric lebih mahal). Fitur kategorikal. |
| **Engine HP**         | Daya mesin; makin tinggi biasanya makin mahal. Fitur numerik penting.                                  |
| **Engine Cylinders**  | Mempengaruhi performa dan konsumsi bahan bakar. Korelasi dengan harga.                                 |
| **Transmission Type** | Preferensi pasar dan kenyamanan berkendara bisa beda berdasarkan jenis transmisi.                      |
| **Driven\_Wheels**    | Sistem penggerak memengaruhi performa di berbagai medan → pengaruh ke harga.                           |
| **Number of Doors**   | Kadang mencerminkan gaya kendaraan (sport vs keluarga). Pengaruh kecil tapi tetap relevan.             |
| **Market Category**   | Segmentasi pasar (misal: luxury, crossover). Bisa langsung terkait dengan positioning harga.           |
| **Vehicle Size**      | Ukuran kendaraan mencerminkan kapasitas dan penggunaan (kompak lebih murah, SUV lebih mahal).          |
| **Vehicle Style**     | Tipe bodi bisa menentukan target pasar dan harga (sedan vs convertible).                               |
| **highway MPG**       | Efisiensi bahan bakar di jalan bebas hambatan → pengaruh ke nilai mobil.                               |
| **city mpg**          | Efisiensi di dalam kota. Digabungkan dengan highway MPG bisa jadi indikator performa mesin.            |
| **Popularity**        | Indikator minat pasar. Mobil populer bisa punya harga lebih tinggi karena permintaan.                  |
| **MSRP**              | Target/label yang ingin diprediksi. Harga jual mobil berdasarkan semua fitur di atas.                  |


Import seluruh library yang dibutuhkan untuk analisis data, visualisasi, preprocessing, pemodelan, dan evaluasi performa model.

In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.pipeline import make_pipeline

import math
from sklearn.metrics import r2_score, mean_squared_error

load dataset, membaca dataset, dan menampilkan 5 baris pertama.

In [2]:
cars_data = pd.read_csv('data.csv')
cars_data.head()

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500


Menampilkan tipe data dan statistik deskriptif dari setiap kolom.

In [3]:
cars_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11914 entries, 0 to 11913
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Make               11914 non-null  object 
 1   Model              11914 non-null  object 
 2   Year               11914 non-null  int64  
 3   Engine Fuel Type   11911 non-null  object 
 4   Engine HP          11845 non-null  float64
 5   Engine Cylinders   11884 non-null  float64
 6   Transmission Type  11914 non-null  object 
 7   Driven_Wheels      11914 non-null  object 
 8   Number of Doors    11908 non-null  float64
 9   Market Category    8172 non-null   object 
 10  Vehicle Size       11914 non-null  object 
 11  Vehicle Style      11914 non-null  object 
 12  highway MPG        11914 non-null  int64  
 13  city mpg           11914 non-null  int64  
 14  Popularity         11914 non-null  int64  
 15  MSRP               11914 non-null  int64  
dtypes: float64(3), int64(5

In [4]:
cars_data.describe()

,Year,Engine HP,Engine Cylinders,Number of Doors,highway MPG,city mpg,Popularity,MSRP
count,11914.000000,11845.00000,11884.000000,11908.000000,11914.000000,11914.000000,11914.000000,1.191400e+04
mean,2010.384338,249.38607,5.628829,3.436093,26.637485,19.733255,1554.911197,4.059474e+04
std,7.579740,109.19187,1.780559,0.881315,8.863001,8.987798,1441.855347,6.010910e+04
min,1990.000000,55.00000,0.000000,2.000000,12.000000,7.000000,2.000000,2.000000e+03
25%,2007.000000,170.00000,4.000000,2.000000,22.000000,16.000000,549.000000,2.100000e+04
50%,2015.000000,227.00000,6.000000,4.000000,26.000000,18.000000,1385.000000,2.999500e+04
75%,2016.000000,300.00000,6.000000,4.000000,30.000000,22.000000,2009.000000,4.223125e+04
max,2017.000000,1001.00000,16.000000,4.000000,354.000000,137.000000,5657.000000,2.065902e+06


Insight yang didapat:


* **Tahun:** Tahun minimum dalam dataset adalah **1990**, rata-rata tahun adalah **2010**, dan tahun maksimum adalah **2017**.

* **Tenaga Mesin (Engine HP):** Tenaga minimum dalam dataset adalah **55 HP**, rata-rata tenaga adalah **249 HP**, dan tenaga maksimum adalah **1001 HP**.

* **Jumlah Silinder (Engine Cylinders):** Jumlah silinder minimum dalam dataset adalah **0**, rata-rata jumlah silinder adalah **5**, dan jumlah silinder maksimum adalah **16**.

* **Jumlah Pintu (Number of Doors):** Jumlah pintu minimum pada mobil adalah **2**, rata-rata jumlah pintu adalah **3**, dan jumlah maksimum adalah **4**.

* **Efisiensi Bahan Bakar di Jalan Tol (Highway MPG):** Efisiensi minimum adalah **12 MPG**, rata-rata adalah **26,6 MPG**, dan efisiensi maksimum adalah **354 MPG**.

* **Efisiensi Bahan Bakar di Kota (City MPG):** Efisiensi minimum adalah **7 MPG**, rata-rata adalah **19,7 MPG**, dan efisiensi maksimum adalah **137 MPG**.

* **Popularitas:** Popularitas minimum dalam dataset adalah **2**, rata-rata popularitas mobil adalah **1.554,9**, dan popularitas maksimum adalah **5.657**.

* **Harga (Price):** Harga mobil minimum dalam dataset adalah **Rp 2.000**, rata-rata harga adalah sekitar **Rp 40.000.000**, dan harga maksimum adalah **Rp 2,06 miliar**.

## Data Cleaning

Melakukan standarisasi nama kolom dan mengganti nama agar lebih mudah dibaca.

In [5]:
cars_data.columns = cars_data.columns.str.lower().str.replace(" ", "_")

cars_data.rename(columns={
    'engine_fuel_type': 'fuel_type',
    'engine_hp': 'hp',
    'engine_cylinders': 'cylinders',
    'transmission_type': 'transmission',
    'driven_wheels': 'drive',
    'number_of_doors': 'doors',
    'market_category': 'market',
    'vehicle_size': 'size',
    'vehicle_style': 'style',
    'msrp': 'price'
}, inplace=True)

In [6]:
print('Jumlah duplikat: ', cars_data.duplicated().sum())
cars_data = cars_data.drop_duplicates()

Jumlah duplikat:  715


In [7]:
print('Jumlah missing value tiap kolom: ')
print(cars_data.isnull().sum())

Jumlah missing value tiap kolom: 
make               0
model              0
year               0
fuel_type          3
hp                69
cylinders         30
transmission       0
drive              0
doors              6
market          3376
size               0
style              0
highway_mpg        0
city_mpg           0
popularity         0
price              0
dtype: int64


Saya hapus kolom market karena kolom ini memiliki terlalu banyak nilai kosong (null) dan juga fitur ini tidak memiliki pengaruh yang signifikan terhadap fitur target, yaitu price (harga).

In [8]:
cars_data.drop('market', axis = 1, inplace = True)

<ipython-input-8-7e4d45260de2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cars_data.drop('market', axis = 1, inplace = True)


check kolom dataset yang punya nilai null

In [9]:
null_values = cars_data[cars_data.isnull().any(axis = 1)]
null_values

,make,model,year,fuel_type,hp,cylinders,transmission,drive,doors,size,style,highway_mpg,city_mpg,popularity,price
539,FIAT,500e,2015,electric,NaN,0.0,DIRECT_DRIVE,front wheel drive,2.0,Compact,2dr Hatchback,108,122,819,31800
540,FIAT,500e,2016,electric,NaN,0.0,DIRECT_DRIVE,front wheel drive,2.0,Compact,2dr Hatchback,103,121,819,31800
541,FIAT,500e,2017,electric,NaN,0.0,DIRECT_DRIVE,front wheel drive,2.0,Compact,2dr Hatchback,103,121,819,31800
1983,Chevrolet,Bolt EV,2017,electric,200.0,NaN,DIRECT_DRIVE,front wheel drive,4.0,Compact,4dr Hatchback,110,128,1385,40905
1984,Chevrolet,Bolt EV,2017,electric,200.0,NaN,DIRECT_DRIVE,front wheel drive,4.0,Compact,4dr Hatchback,110,128,1385,36620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9853,Kia,Soul EV,2016,electric,NaN,0.0,DIRECT_DRIVE,front wheel drive,4.0,Compact,Wagon,92,120,1720,31950
9854,Kia,Soul EV,2016,electric,NaN,0.0,DIRECT_DRIVE,front wheel drive,4.0,Compact,Wagon,92,120,1720,35950
11321,Suzuki,Verona,2004,NaN,155.0,6.0,AUTOMATIC,front wheel drive,4.0,Midsize,Sedan,25,17,481,17199
11322,Suzuki,Verona,2004,NaN,155.0,6.0,AUTOMATIC,front wheel drive,4.0,Midsize,Sedan,25,17,481,20199


**Kolom Fuel Type, HP, Cylinders, dan Doors memiliki nilai kosong (null).**

* Nilai kosong pada kolom **fuel\_type** akan diisi dengan **modus (mode)** karena kolom ini bersifat **kategorikal**.
* Kolom **hp** akan diisi dengan **nilai 0**, karena mobil yang memiliki nilai kosong adalah **mobil listrik**, yang umumnya memang **tidak memiliki tenaga mesin (HP)**.
* Karena **mobil listrik juga tidak memiliki silinder**, maka nilai kosong pada kolom **cylinders** juga akan diisi dengan **0**.
* Untuk kolom **doors**, nilai kosong akan diisi dengan **rata-rata (mean)** karena kolom ini bersifat **numerik**.

Mengisi missing value dengan strategi yang sesuai jenis data.

In [10]:
cars_data['fuel_type'] = cars_data['fuel_type'].fillna('regular unleaded')
cars_data['hp'] = cars_data['hp'].fillna(0)
cars_data['cylinders'] = cars_data['cylinders'].fillna(0)
cars_data['doors'] = cars_data['doors'].fillna(cars_data['doors'].mean())

In [11]:
num_col = cars_data.select_dtypes(include = [np.number])
cat_col = cars_data.select_dtypes(exclude = [np.number])

Menghapus Data dengan Label UNKNOWN

In [12]:
cars_data.drop(cars_data[cars_data['transmission']=='UNKNOWN'].index, axis='index', inplace = True)

### Handling Outliers

Visualisasi outlier menggunakan boxplot.

In [13]:
for i in num_col:
    fig = px.box(cars_data, x = cars_data[i])
    fig.update_traces(fillcolor = '#C9A26B')
    fig.show()

Insight:
* **Tenaga Mesin (Hp)**: Tenaga mesin di atas 500 dianggap sebagai pencilan (outlier).
* **Jumlah Silinder Mesin**: Jumlah silinder di atas 8 dan di bawah 3 dianggap sebagai pencilan.
* **Highway MPG (miles per gallon)**: MPG di jalan raya yang lebih rendah dari 12 atau lebih tinggi dari 42 dianggap sebagai pencilan.
* **City MPG (miles per gallon)**: MPG di kota yang lebih rendah dari 7 atau lebih tinggi dari 31 dianggap sebagai pencilan.
* **Harga**: Harga mobil yang lebih rendah dari 2.000 atau lebih tinggi dari 70,9K dianggap sebagai pencilan.

In [14]:
s1 = cars_data.shape
clean = cars_data[['hp', 'cylinders', 'highway_mpg', 'city_mpg', 'price']]
for i in clean.columns:
    qt1 = cars_data[i].quantile(0.25)
    qt3 = cars_data[i].quantile(0.75)
    iqr =  qt3 - qt1
    lower = qt1-(1.5*iqr)
    upper = qt3+(1.5*iqr)
    min_in = cars_data[cars_data[i]<lower].index
    max_in = cars_data[cars_data[i]>upper].index
    cars_data.drop(min_in, inplace = True)
    cars_data.drop(max_in, inplace = True)
s2 = cars_data.shape
outliers = s1[0] - s2[0]
print("Outliers yang terhapus: ", outliers)

Outliers yang terhapus:  1403


In [15]:
fig = px.box(cars_data, x = cars_data['hp'])
fig.update_traces(fillcolor = '#C9A26B')

In [16]:
cars_data.describe()

,year,hp,cylinders,doors,highway_mpg,city_mpg,popularity,price
count,9784.00000,9784.000000,9784.000000,9784.000000,9784.000000,9784.000000,9784.000000,9784.000000
mean,2010.31204,231.154334,5.424162,3.505110,26.304374,19.180192,1560.331664,29204.537612
std,7.46610,78.693348,1.395065,0.841754,5.745849,4.376362,1464.248520,15605.472026
min,1990.00000,0.000000,3.000000,2.000000,12.000000,10.000000,21.000000,2000.000000
25%,2006.00000,170.000000,4.000000,3.000000,22.000000,16.000000,549.000000,20498.000000
50%,2014.00000,220.000000,6.000000,4.000000,26.000000,18.000000,1385.000000,29089.000000
75%,2016.00000,288.000000,6.000000,4.000000,30.000000,22.000000,2009.000000,38850.000000
max,2017.00000,485.000000,8.000000,4.000000,42.000000,31.000000,5657.000000,70900.000000


Insight:
* **Tenaga Mesin (Hp)**: Setelah penghapusan outlier, tenaga mesin minimum adalah 0, rata-rata 231, dan maksimum 485.
* **Jumlah Silinder Mesin**: Setelah penghapusan outlier, jumlah silinder minimum adalah 3, rata-rata 5, dan maksimum 8.
* **Highway MPG (miles per gallon)**: Setelah penghapusan outlier, nilai minimum adalah 12, rata-rata 26,3, dan maksimum 42.
* **City MPG (miles per gallon)**: Setelah penghapusan outlier, nilai minimum adalah 10, rata-rata 19,1, dan maksimum 31.
* **Harga**: Setelah penghapusan outlier, harga mobil minimum adalah 2.000, rata-rata sekitar 29.000, dan maksimum 70.900.

### Univariate Analysis

In [17]:
for i in cars_data:
    fig = px.histogram(cars_data, x= i, color_discrete_sequence = ['#C9A26B'])
    fig.show()

Insight:
* **Merek (Make)**: Jumlah paling sedikit: *Maserati* (2), jumlah terbanyak: *Chevrolet* (1039).
* **Model**: Jumlah paling sedikit: banyak model hanya muncul sekali (1), jumlah terbanyak: *Silverado 1500* (156).
* **Tahun (Year)**: Jumlah paling sedikit: *1990* (76), jumlah terbanyak: *2016* (1756).
* **Jenis Bahan Bakar (Fuel Type)**: Jumlah paling sedikit: *Natural Gas* (2), jumlah terbanyak: *Regular Unleaded* (6380).
* **Tenaga Mesin (Hp)**: Jumlah paling sedikit: banyak nilai hp hanya muncul sekali (1), jumlah terbanyak: *200 hp* (355).
* **Jumlah Silinder (Cylinders)**: Jumlah paling sedikit: *3 silinder* (11), jumlah terbanyak: *4 silinder* (4110).
* **Transmisi (Transmission)**: Jumlah paling sedikit: *Automated Manual* (316), jumlah terbanyak: *Automatic* (7097).
* **Sistem Penggerak (Drive)**: Jumlah paling sedikit: *four wheel drive* (1220), jumlah terbanyak: *front wheel drive* (4056).
* **Jumlah Pintu (Doors)**: Jumlah paling sedikit: *3 pintu* (356), jumlah terbanyak: *4 pintu* (7185).
* **Ukuran (Size)**: Jumlah paling sedikit: *Large* (2241), jumlah terbanyak: *Compact* (3822).
* **Gaya (Style)**: Jumlah paling sedikit: *Convertible SUV* (28), jumlah terbanyak: *Sedan* (2448).
* **Highway MPG**: Jumlah paling sedikit: *12 mpg* (1), jumlah terbanyak: *24 mpg* (752).
* **City MPG**: Jumlah paling sedikit: *10 mpg* (5), jumlah terbanyak: *17 mpg* (1117).
* **Popularitas (Popularity)**: Jumlah paling sedikit: skor *238* (2), jumlah terbanyak: skor *1385* (1039).
* **Harga (Price)**: Jumlah paling sedikit: banyak harga hanya muncul sekali (1), jumlah terbanyak: *2000* (736).

### Bivariate Analysis

In [19]:
categorical_columns = ['model', 'fuel_type', 'transmission', 'drive', 'style', 'size']

for col in categorical_columns:
    try:
        top_hp = cars_data.groupby(col)['hp'].mean().sort_values(ascending=False).head(5).reset_index()

        fig = px.bar(top_hp,
                     x=col,
                     y='hp',
                     title=f'Top 5 {col} with highest average Hp',
                     labels={col: col.capitalize(), 'hp': 'Hp'},
                     color_discrete_sequence=['#C49A6C'])

        fig.show()
    except Exception as e:
        print(f"Error processing {col}: {e}")

Insight:
* **Merek:** Merek mobil Chrysler memiliki konsumsi bahan bakar dalam kota (city mpg) tertinggi yaitu 16.6.
* **Model:** Model mobil 190-Class dan beberapa lainnya memiliki konsumsi bahan bakar dalam kota terendah yaitu 11.
* **Tipe Bahan Bakar:** Tipe bahan bakar Diesel memiliki konsumsi bahan bakar dalam kota tertinggi yaitu 18.5.
* **Transmisi:** Transmisi Manual memiliki konsumsi bahan bakar dalam kota terendah yaitu 18.6, sedangkan transmisi Automated-Manual memiliki konsumsi tertinggi yaitu 23.8.
* **Sistem Penggerak:** Sistem penggerak roda belakang (Rear wheel drive) memiliki konsumsi dalam kota terendah yaitu 15.1, sedangkan penggerak empat roda (Four wheel drive) memiliki konsumsi tertinggi yaitu 21.8.
* **Ukuran Mobil:** Mobil berukuran Compact memiliki konsumsi dalam kota terendah yaitu 15.8, sedangkan mobil berukuran Large memiliki konsumsi tertinggi yaitu 21.3.
* **Gaya Mobil (Style):** Mobil dengan gaya Coupe memiliki konsumsi dalam kota terendah yaitu 12.5, sedangkan Passenger Van memiliki konsumsi tertinggi yaitu 24.3.

Relasi antara label harga (price) dan fitur lainnya

In [20]:
numerical_columns = ['year', 'hp', 'cylinders', 'doors', 'city_mpg', 'highway_mpg']

for col in numerical_columns:
  fig = px.scatter(cars_data, x = col, y = 'price', color = 'cylinders')
  fig.show()

**Insights**

* Sebagian besar mobil dengan popularitas 204 dan 6 silinder memiliki harga mulai dari 35K hingga mencapai 57K.
* Mobil dengan popularitas 549 dan 8 silinder memiliki harga mulai dari 42K hingga mencapai 70.2K.
* Mobil dengan popularitas 1385 dan 8 silinder memiliki harga di atas 44K.
* Mobil dengan popularitas 5657 dan harga di atas 21K lebih cenderung memiliki 6 silinder.

## Preprocessing

Melakukan encoding terhadap variabel kategorikal, split dataset menjadi train dan test, serta standarisasi fitur.

In [21]:
cat_features = ['make', 'model', 'fuel_type', 'transmission', 'drive', 'size', 'style']
cars_data = pd.get_dummies(cars_data, columns = cat_features)

In [22]:
X = cars_data.drop('price', axis = 1)
y = cars_data['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [23]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Modelling

#### Random Forest Regressor

Melatih model Random Forest Regressor dan mengukur performa menggunakan R² dan RMSE.

In [33]:
rfr = RandomForestRegressor(n_estimators = 40)
rfr_algo = make_pipeline(rfr)

rfr_algo.fit(X_train, y_train)
rfr_pred = rfr_algo.predict(X_test)

rfr_r2 = r2_score(y_test, rfr_pred)
rfr_rmse = math.sqrt(mean_squared_error(y_test, rfr_pred))

In [34]:
print('R2 Score (RFR): ', rfr_r2)
print('RMSE (RFR): ', rfr_rmse)

R2 Score (RFR):  0.9572673140635594
RMSE (RFR):  3335.1476547398415


#### Linear Regression

Melatih model regresi linier sebagai baseline dan mengukur performa.

In [36]:
lr = LinearRegression()
lr_algo = make_pipeline(lr)

lr_algo.fit(X_train, y_train)
lr_pred = lr_algo.predict(X_test)

lr_r2 = r2_score(y_test, lr_pred)
lr_rmse = math.sqrt(mean_squared_error(y_test, lr_pred))

In [37]:
print('R2 Score (Linear Regression):', lr_r2)
print('RMSE (Linear Regression):', lr_rmse)

R2 Score (Linear Regression): 0.9516060077164159
RMSE (Linear Regression): 3549.2018225540005


#### Gradient Booster Regressor

Melatih model Gradient Boosting dan mengevaluasi performanya.

In [38]:
gbr = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=1)
gbr_algo = make_pipeline(gbr)

gbr_algo.fit(X_train, y_train)
gbr_pred = gbr_algo.predict(X_test)

gbr_r2 = r2_score(y_test, gbr_pred)
gbr_rmse = math.sqrt(mean_squared_error(y_test, gbr_pred))

In [39]:
print('R2 Score (Gradient Boosting):', gbr_r2)
print('RMSE (Gradient Boosting):', gbr_rmse)

R2 Score (Gradient Boosting): 0.9228962899620576
RMSE (Gradient Boosting): 4479.944237136683


Membandingkan performa semua model secara numerik.

In [40]:
metrics = pd.DataFrame({
    'Model': ['Random Forest Regressor', 'Linear Regression', 'Gradient Boosting'],
    'R2 Score': [rfr_r2, lr_r2, gbr_r2],
    'RMSE': [rfr_rmse, lr_rmse, gbr_rmse]
})
metrics

,Model,R2 Score,RMSE
0,Random Forest Regressor,0.957267,3335.147655
1,Linear Regression,0.951606,3549.201823
2,Gradient Boosting,0.922896,4479.944237


### Kelebihan dan Kekurangan Model

#### 1. **Random Forest Regressor (RFR)**

* **Kelebihan:**

  * Dapat menangani hubungan non-linier antara fitur dan target.
  * Tidak sensitif terhadap outlier.
  * Dapat menangani data dengan banyak fitur dan kompleksitas yang tinggi.
  * Memberikan hasil yang stabil dan dapat diandalkan pada data yang beragam.
* **Kekurangan:**

  * Lebih lambat dalam prediksi dibandingkan regresi linier.
  * Model yang lebih kompleks dan sulit untuk interpretasi.
  * Mungkin overfitting pada data yang sangat besar atau sangat kompleks.

#### 2. **Linear Regression (LR)**

* **Kelebihan:**

  * Sederhana dan cepat.
  * Mudah dipahami dan diinterpretasikan.
  * Baik digunakan untuk data dengan hubungan linier.
  * Tidak memerlukan banyak tuning parameter.
* **Kekurangan:**

  * Hanya efektif untuk data yang memiliki hubungan linier antara fitur dan target.
  * Rentan terhadap overfitting jika jumlah fitur sangat besar atau data memiliki multikolinearitas tinggi.
  * Tidak bisa menangani data yang sangat kompleks atau data dengan outlier signifikan.

#### 3. **Gradient Boosting Regressor (GBR)**

* **Kelebihan:**

  * Dapat menangani hubungan non-linier dengan sangat baik.
  * Cenderung memberikan hasil yang sangat baik di banyak kasus dengan tunning parameter yang tepat.
  * Mengatasi masalah overfitting lebih baik dibandingkan Random Forest.
* **Kekurangan:**

  * Lebih lambat dan memerlukan lebih banyak waktu untuk pelatihan dibandingkan model lain.
  * Perlu tuning parameter yang cukup mendalam untuk mencapai hasil terbaik.
  * Dapat rentan terhadap overfitting jika tidak hati-hati dengan hyperparameter tuning.

### Pilihan Model Terbaik

Berdasarkan hasil evaluasi ini, Random Forest Regressor adalah model yang memberikan performa terbaik di antara ketiganya, dengan R² Score tertinggi dan RMSE terendah. Linear Regression juga memiliki performa yang sangat baik, meskipun sedikit lebih rendah dibandingkan Random Forest. Gradient Boosting, meskipun sering kali kuat dalam banyak kasus, tidak memberikan hasil yang sebaik dua model lainnya dalam hal ini.